# Web scraping - transfers/contracts

In [2]:
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import requests
import pandas as pd
import requests

In [3]:
load_dotenv()

def get_soup(url):
    page = requests.get(url,headers = {"User-Agent":"Mozilla/5.0","origin":"https://www.transfermarkt.co.uk"})
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

In [6]:
player_urls = []
for year in ['2018','2019','2020']:
    transfer_soup = get_soup(f'https://www.transfermarkt.co.uk/premier-league/transfers/wettbewerb/GB1?saison_id={year}')
    main_div = transfer_soup.find('div',{'class':'large-8 columns'})
    for team in main_div.find_all('div',{'class':'box'})[3:]:
        team_name = team.find('div',{'class':'table-header'}).find('h2')
        transfers = team.find_all('div',{'class':'responsive-table'})
        transfer_table = transfers[0].find('tbody')
        for transfer_table in transfers[0:1]:
            transfer_table = transfer_table.find('tbody')
            for tr in transfer_table.find_all('tr'):
                td = tr.find('td')
                span = td.find('div').find('span')
                link = span.find('a', {'class':'spielprofil_tooltip'})
                player_urls.append([link.text,link["href"]])


In [9]:
def get_player_transfers(player):
    soup_url = player[1].replace('/profil/','/transfers/')
    player_soup = get_soup('https://www.transfermarkt.co.uk'+soup_url)
    transfer_history = player_soup.find_all('tr', {'class':'zeile-transfer'})
    transfer_list = []
    for tr in transfer_history:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td if tr.text.strip() != '']
        row.append(player[0])
        transfer_list.append(row)
        
    df = pd.DataFrame(transfer_list, columns = ["season","date","left","joined","mv","fee","name"])
    print(player[0] + ' added')
    return df

In [ ]:
## import time
import random

df = pd.DataFrame([], columns = ["season","date","left","joined","mv","fee","name"])
for player in player_urls:
    player_df = get_player_transfers(player)
    df = pd.concat([df,player_df])
    time.sleep(random.randint(15, 26)) # to not break their website

In [12]:
df.to_csv('./CSVs/transfers.csv')